# General Imports and Pre-Requisites

In [1]:
import carla
import random
import numpy as np
import cv2
import queue

from carla import Transform, Location
from utils import download_installer, get_actor_blueprints, remove_actors

import settings

settings.init()
download_installer()

# Connect to the client and get the world object
client = carla.Client("localhost", settings.carla_port)
world = client.get_world()
spectator = world.get_spectator()

# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

all_id = []

# Adoption of the World to meet our Visualisation needs

In [ ]:
# List possible worlds
#print(client.get_available_maps())
#client.load_world('Town04') # larger towns with crossing Town03, Town05

In [2]:
# remove trees to have a better view
world.unload_map_layer(carla.MapLayer.Foliage)

# Add Camera

In [5]:
cameras = []
camera_data = {}

def camera_callback(camera):
    state = camera.id

    def process(image):
        if state in camera_data:
            image.save_to_disk('output/' + str(state) + '/' + '%.6d.jpg' % image.frame)
            camera_data[state].put(np.reshape(
                np.copy(image.raw_data), (image.height, image.width, 4)
            ))

    return process

for i in range(settings.number_of_cameras):
    if i >= settings.number_of_cameras:
        break
    else:
        input("Press Enter to continue...")

    spectator_location = world.get_spectator().get_transform()
    camera_bp = bp_lib.find("sensor.camera.rgb")

    camera = world.spawn_actor(camera_bp, spectator_location)
    cameras.append(camera)

    # Get gamera dimensions and initialise dictionary
    image_w = camera_bp.get_attribute("image_size_x").as_int()
    image_h = camera_bp.get_attribute("image_size_y").as_int()
    camera_data[camera.id] = queue.Queue()
    camera_data[camera.id].put(np.zeros((image_h, image_w, 4)))

    # Start camera recording
    callback = camera_callback(camera)
    camera.listen(callback)

# Game loop
while True:
    for camera in cameras:
        if camera.id in camera_data and not camera_data[camera.id].empty():
            cv2.imshow("RGB Camera " + str(camera.id), camera_data[camera.id].get())

    # Quit if user presses 'esc'
    key = cv2.waitKey(1)
    if key == 27:
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()

# Add Vehicle

In [7]:
# Add traffic to the simulation
while world.get_actors().filter("*vehicle*").__len__() < settings.number_of_vehicles:
    vehicle_bp = random.choice(bp_lib.filter("vehicle"))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

    if npc is not None:
        print("Vehicle {} spawned".format(npc.id))

# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter("*vehicle*"):
    v.set_autopilot(True)

Vehicle 220 spawned
Vehicle 221 spawned
Vehicle 222 spawned
Vehicle 223 spawned
Vehicle 224 spawned
Vehicle 225 spawned
Vehicle 226 spawned
Vehicle 227 spawned
Vehicle 228 spawned
Vehicle 229 spawned
Vehicle 230 spawned
Vehicle 231 spawned
Vehicle 232 spawned
Vehicle 233 spawned
Vehicle 234 spawned
Vehicle 235 spawned
Vehicle 236 spawned
Vehicle 237 spawned
Vehicle 238 spawned
Vehicle 239 spawned
Vehicle 240 spawned
Vehicle 241 spawned
Vehicle 242 spawned
Vehicle 243 spawned
Vehicle 244 spawned
Vehicle 245 spawned
Vehicle 246 spawned
Vehicle 247 spawned
Vehicle 248 spawned
Vehicle 249 spawned
Vehicle 250 spawned
Vehicle 251 spawned
Vehicle 252 spawned
Vehicle 253 spawned
Vehicle 254 spawned
Vehicle 255 spawned
Vehicle 256 spawned
Vehicle 257 spawned
Vehicle 258 spawned
Vehicle 259 spawned
Vehicle 260 spawned
Vehicle 261 spawned
Vehicle 262 spawned
Vehicle 263 spawned
Vehicle 264 spawned
Vehicle 265 spawned
Vehicle 266 spawned
Vehicle 267 spawned
Vehicle 268 spawned
Vehicle 269 spawned


# Add Pedestrians

In [8]:
SpawnActor = carla.command.SpawnActor
walkers_list = []
all_id = []

# 2. we spawn the walker object
batch = []
walker_speed = []
for i in range(settings.number_of_walkers):
    spawn_point = random.choice(spawn_points)
    walker_bp = random.choice(get_actor_blueprints(world, "walker.pedestrian.*", "all"))
    # set as not invincible
    if walker_bp.has_attribute("is_invincible"):
        walker_bp.set_attribute("is_invincible", "false")
    # set the max speed
    if walker_bp.has_attribute("speed"):
        if random.random() > settings.percentage_pedestrians_running:
            # walking
            walker_speed.append(walker_bp.get_attribute("speed").recommended_values[1])
        else:
            # running
            walker_speed.append(walker_bp.get_attribute("speed").recommended_values[2])
    else:
        print("Walker has no speed")
        walker_speed.append(0.0)
    batch.append(SpawnActor(walker_bp, spawn_point))
results = client.apply_batch_sync(batch, True)
walker_speed2 = []
for i in range(len(results)):
    if results[i].error:
        print(results[i].error)
    else:
        walkers_list.append({"id": results[i].actor_id})
        walker_speed2.append(walker_speed[i])
walker_speed = walker_speed2

# 3. we spawn the walker controller
batch = []
walker_controller_bp = world.get_blueprint_library().find("controller.ai.walker")
for i in range(len(walkers_list)):
    batch.append(
        SpawnActor(walker_controller_bp, carla.Transform(), walkers_list[i]["id"])
    )
results = client.apply_batch_sync(batch, True)
for i in range(len(results)):
    if results[i].error:
        print(results[i].error)
    else:
        walkers_list[i]["con"] = results[i].actor_id

# 4. we put together the walkers and controllers id to get the objects from their id
for i in range(len(walkers_list)):
    all_id.append(walkers_list[i]["con"])
    all_id.append(walkers_list[i]["id"])
all_actors = world.get_actors(all_id)
world.tick()

# 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
# set how many pedestrians can cross the road
world.set_pedestrians_cross_factor(settings.percentage_pedestrians_crossing)
for i in range(0, len(all_id), 2):
    all_actors[i].start()

    # set walk to random point
    all_actors[i].go_to_location(world.get_random_location_from_navigation())
    # max speed
    all_actors[i].set_max_speed(float(walker_speed[int(i / 2)]))

Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position


# Clear Scene

In [9]:
# Stop the camera when we've recorded enough data
[camera.stop() for camera in cameras]
[remove_actors(world, actor) for actor in ["walker", "vehicle"]]

Removing walker 393
Removing walker 387
Removing walker 397
Removing walker 407
Removing walker 401
Removing walker 395
Removing walker 405
Removing walker 415
Removing walker 409
Removing walker 403
Removing walker 413
Removing walker 356
Removing walker 336
Removing walker 342
Removing walker 412
Removing walker 411
Removing walker 410
Removing walker 408
Removing walker 406
Removing walker 404
Removing walker 402
Removing walker 400
Removing walker 399
Removing walker 398
Removing walker 396
Removing walker 394
Removing walker 392
Removing walker 391
Removing walker 390
Removing walker 389
Removing walker 388
Removing walker 386
Removing walker 385
Removing walker 384
Removing walker 382
Removing walker 380
Removing walker 379
Removing walker 377
Removing walker 375
Removing walker 374
Removing walker 372
Removing walker 371
Removing walker 369
Removing walker 367
Removing walker 366
Removing walker 364
Removing walker 363
Removing walker 361
Removing walker 360
Removing walker 358


: 

: 